In [ ]:
from biber_manager.download import fetch_biber_biblatex_version_table

In [31]:

from __future__ import annotations

import re

from requests_html import HTMLSession

BASE_URL = "https://sourceforge.net"
FILES_URL = f"{BASE_URL}/projects/biblatex-biber/files/biblatex-biber"
DOC_TEX_URL = "https://raw.githubusercontent.com/plk/biber/dev/doc/biber.tex"

VALID_VERSION_PATTERN = re.compile(r"^(\d+\.\d+(\.\d+)?|current)$")
VERSION_TABLE_BODY_PATTER = re.compile(
    r"\\midrule\n(?P<rows>.+?)\\bottomrule.+\\caption\{\\biber\/\\biblatex\\ compatibility matrix",
    re.DOTALL,
)
VERSION_TABLE_VERSION_PATTER = re.compile(
    r"(?P<biber_version>\d\.[.\d]+) & (?P<biblatex_version>\d\.[.\d]+x?(?:, \d\.[.\d]+x?)?)\\\\\n",
    re.DOTALL,
)

VERSION_DOWNLOAD_EXTENSION = {
    "win32": "binaries/Windows/biber-MSWIN64.zip",
    "cygwin": "binaries/Cygwin/biber-cygwin64.tar.gz",
    "linux": "binaries/Linux/biber-linux_x86_64.tar.gz",
    "darwin": "binaries/MacOS/biber-darwin_universal.tar.gz",
    "freebsd": "binaries/FreeBSD/biber-amd64-freebsd.tar.xz",
}


def is_valid_version(version_str: str) -> bool:
    """Verify that a version string has a valid value.

    Parameters
    ----------
    version_str: str
        Version string to validate.

    Returns
    -------
    bool
        Whether or not the version string is valid.
    """
    return VALID_VERSION_PATTERN.match(version_str) is not None


def find_valid_biber_versions() -> dict[str, str]:
    """Find available biber versions on sourceforge.

    Returns
    -------
    dict[str, str]
        Valid versions with version string as key and partial download paths as value.

    See Also
    --------
    is_valid_version
    """
    version_dict = {}
    session = HTMLSession()
    resp = session.get(FILES_URL)
    for link in resp.html.find("tr.folder th a"):
        version_span = link.find("span", first=True)
        if version_span is not None:
            version_str = version_span.text
            if is_valid_version(version_str) is True:
                version_dict[version_str] = f"{BASE_URL}{link.attrs['href']}"

    return version_dict


def fetch_biber_biblatex_version_table() ->list[dict[str, str]]:

    session = HTMLSession()
    resp = session.get(DOC_TEX_URL)

    rows_match = VERSION_TABLE_BODY_PATTER.search(resp.text)
    if rows_match is None:
      raise ValueError("Could not find table pattern.")
    
    return [row.groupdict() for row in VERSION_TABLE_VERSION_PATTER.finditer(rows_match["rows"])]


In [32]:

session = HTMLSession()
resp = session.get(DOC_TEX_URL)

row_match = VERSION_TABLE_BODY_PATTER.search(resp.text)


In [18]:
print(row_match["rows"])

2.18 & 3.18\\
2.17 & 3.17\\
2.16 & 3.16\\
2.15 & 3.15\\
2.14 & 3.14\\
2.13 & 3.13\\
2.12 & 3.12\\
2.11 & 3.11\\
2.10 & 3.10\\
2.9 & 3.9\\
2.8 & 3.8\\
2.7 & 3.7\\
2.6 & 3.5, 3.6\\
2.5 & 3.4\\
2.4 & 3.3\\
2.3 & 3.2\\
2.2 & 3.1\\
2.1 & 3.0\\
2.0 & 3.0\\
1.9 & 2.9\\
1.8 & 2.8x\\
1.7 & 2.7\\
1.6 & 2.6\\
1.5 & 2.5\\
1.4 & 2.4\\
1.3 & 2.3\\
1.2 & 2.1, 2.2\\
1.1 & 2.1\\
1.0 & 2.0\\
0.9.9 & 1.7x\\
0.9.8 & 1.7x\\
0.9.7 & 1.7x\\
0.9.6 & 1.7x\\
0.9.5 & 1.6x\\
0.9.4 & 1.5x\\
0.9.3 & 1.5x\\
0.9.2 & 1.4x\\
0.9.1 & 1.4x\\
0.9 & 1.4x\\



In [33]:
row = VERSION_TABLE_VERSION_PATTER.finditer(row_match["rows"])

In [38]:
versions = [row.groupdict() for row in VERSION_TABLE_VERSION_PATTER.finditer(row_match["rows"])]

In [40]:
from rich.console import Console
from rich.table import Table

table = Table(title="Version compatibility")

table.add_column("Biber", justify="right", style="cyan", no_wrap=True)
table.add_column("Biblatex", style="magenta", justify="right")

for row in versions:
    table.add_row(row["biber_version"],row["biblatex_version"])

console = Console()
console.print(table)

      Version       
   compatibility    
┏━━━━━━━┳━━━━━━━━━━┓
┃ Biber ┃ Biblatex ┃
┡━━━━━━━╇━━━━━━━━━━┩
│  2.18 │     3.18 │
│  2.17 │     3.17 │
│  2.16 │     3.16 │
│  2.15 │     3.15 │
│  2.14 │     3.14 │
│  2.13 │     3.13 │
│  2.12 │     3.12 │
│  2.11 │     3.11 │
│  2.10 │     3.10 │
│   2.9 │      3.9 │
│   2.8 │      3.8 │
│   2.7 │      3.7 │
│   2.6 │ 3.5, 3.6 │
│   2.5 │      3.4 │
│   2.4 │      3.3 │
│   2.3 │      3.2 │
│   2.2 │      3.1 │
│   2.1 │      3.0 │
│   2.0 │      3.0 │
│   1.9 │      2.9 │
│   1.8 │     2.8x │
│   1.7 │      2.7 │
│   1.6 │      2.6 │
│   1.5 │      2.5 │
│   1.4 │      2.4 │
│   1.3 │      2.3 │
│   1.2 │ 2.1, 2.2 │
│   1.1 │      2.1 │
│   1.0 │      2.0 │
│ 0.9.9 │     1.7x │
│ 0.9.8 │     1.7x │
│ 0.9.7 │     1.7x │
│ 0.9.6 │     1.7x │
│ 0.9.5 │     1.6x │
│ 0.9.4 │     1.5x │
│ 0.9.3 │     1.5x │
│ 0.9.2 │     1.4x │
│ 0.9.1 │     1.4x │
│   0.9 │     1.4x │
└───────┴──────────┘